In [4]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# os.environ을 사용하여 환경 변수를 가져옵니다.
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

if not TOGETHER_API_KEY:
    raise ValueError("TOGETHER_API_KEY 환경 변수가 설정되어 있지 않습니다.")

print(TOGETHER_API_KEY[:10]) # 테스트 후 삭제

tgp_v1_7Ft


In [5]:
from together import Together

# size: small(256*256), default(512*512), large(1024*1024)
def create_image(prompt: str, size: str = "default") -> any:
    client = Together(
        api_key=TOGETHER_API_KEY,
    )
    size_list = {
        "small": [256, 256],
        "default": [512, 512],
        "large": [1024, 1024]
    }

    """
    Organization	    Model Name	                Model String for API	                Default steps   pay($)
    Black Forest Labs	Flux.1 [schnell] (free)*	black-forest-labs/FLUX.1-schnell-Free	N/A             N/A
    Black Forest Labs	Flux.1 [schnell] (Turbo)	black-forest-labs/FLUX.1-schnell	    4               0.0027
    Black Forest Labs	Flux.1 Dev	                black-forest-labs/FLUX.1-dev	        28              0.025
    Black Forest Labs	Flux.1 Canny	            black-forest-labs/FLUX.1-canny*	        28              0.025
    Black Forest Labs	Flux.1 Depth	            black-forest-labs/FLUX.1-depth*	        28              0.025
    Black Forest Labs	Flux.1 Redux	            black-forest-labs/FLUX.1-redux*	        28              0.025
    Black Forest Labs	Flux1.1 [pro]	            black-forest-labs/FLUX.1.1-pro	        -               0.04
    Black Forest Labs	Flux.1 [pro]	            black-forest-labs/FLUX.1-pro	        28              0.05
    Black Forest Labs	Flux .1 Kontext [pro]	    black-forest-labs/FLUX.1-kontext-pro	28              0.04
    Black Forest Labs	Flux .1 Kontext [max]	    black-forest-labs/FLUX.1-kontext-max	28              0.08
    Black Forest Labs	Flux .1 Kontext [dev]	    black-forest-labs/FLUX.1-kontext-dev	28              0.025
    """

    # model_name, steps
    models = {
        "free": ["FLUX.1-schnell-Free", None],
        "schnell": ["FLUX.1-schnell", 4],
        "dev": ["FLUX.1-dev", 28],
        "kontext-dev": ["FLUX.1-kontext-dev", 28],
    }

    model_key = "free"

    response = client.images.generate(
        model=f"black-forest-labs/{models[model_key][0]}",
        steps=models[model_key][1],
        prompt=prompt,
        width=size_list[size][0],
        height=size_list[size][1]
    )

    return response


In [6]:
res = create_image(prompt="Speaker image, product model shot")

In [7]:
res

ImageResponse(id='o57ok3P-2kFHot-96734f814d9d723a-PDX', model='black-forest-labs/FLUX.1-schnell-Free', object='list', data=[ImageChoicesData(index=0, b64_json=None, url='https://api.together.ai/shrt/AAekTe5LN0SUAMGK', timings={'inference': 0.479994114022702})])

In [8]:
# size: small(256*256), default(512*512), large(1024*1024)
def reshape_image(prompt: str, image_url: str, size: str = "default") -> any:
    client = Together(
        api_key=TOGETHER_API_KEY,
    )
    size_list = {
        "small": [256, 256],
        "default": [512, 512],
        "large": [1024, 1024]
    }

    """
    Organization	    Model Name	                Model String for API	                Default steps   pay($)
    Black Forest Labs	Flux.1 [schnell] (free)*	black-forest-labs/FLUX.1-schnell-Free	N/A             N/A
    Black Forest Labs	Flux.1 [schnell] (Turbo)	black-forest-labs/FLUX.1-schnell	    4               0.0027
    Black Forest Labs	Flux.1 Dev	                black-forest-labs/FLUX.1-dev	        28              0.025
    Black Forest Labs	Flux.1 Canny	            black-forest-labs/FLUX.1-canny*	        28              0.025
    Black Forest Labs	Flux.1 Depth	            black-forest-labs/FLUX.1-depth*	        28              0.025
    Black Forest Labs	Flux.1 Redux	            black-forest-labs/FLUX.1-redux*	        28              0.025
    Black Forest Labs	Flux1.1 [pro]	            black-forest-labs/FLUX.1.1-pro	        -               0.04
    Black Forest Labs	Flux.1 [pro]	            black-forest-labs/FLUX.1-pro	        28              0.05
    Black Forest Labs	Flux .1 Kontext [pro]	    black-forest-labs/FLUX.1-kontext-pro	28              0.04
    Black Forest Labs	Flux .1 Kontext [max]	    black-forest-labs/FLUX.1-kontext-max	28              0.08
    Black Forest Labs	Flux .1 Kontext [dev]	    black-forest-labs/FLUX.1-kontext-dev	28              0.025
    """

    # model_name, steps
    models = {
        "kontext-dev": ["FLUX.1-kontext-dev", 28],
    }

    model_key = "kontext-dev"

    response = client.images.generate(
        model=f"black-forest-labs/{models[model_key][0]}",
        steps=models[model_key][1],
        prompt=prompt,
        width=size_list[size][0],
        height=size_list[size][0],
        image_url=image_url,
    )

    return response


In [9]:
r_res = reshape_image(prompt="Natural background placement and situation to highlight the product, product model shot", image_url=res.data[0].url)

In [8]:
import requests

# 확장자 제외한 이름 사용
def download_image(url: str, filename: str | None = None, path="./_data/images") -> any:
    last_part = url.split('/')[-1]
    filename = filename if filename != None else last_part
    filepath = f'{path}/{filename}.jpg'

    try:
        response = requests.get(url)
        # 요청 성공 여부 확인
        if response.status_code == 200:
            # 'wb' (write binary) 모드로 파일을 열고 이미지 데이터를 씀
            with open(filepath, 'wb') as f:
                f.write(response.content)
            print(f"이미지 다운로드 완료: {filepath}")
        else:
            print(f"오류 발생: HTTP {response.status_code}")
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")

In [9]:
download_image(url=res.data[0].url)

이미지 다운로드 완료: ./_data/images/ceHHMdN8AxgCnYxD.jpg


In [10]:
download_image(url=r_res.data[0].url)

이미지 다운로드 완료: ./_data/images/Ei70dDXI2xkxgjVP.jpg
